# Probabilistic Learning Task - PsychoPy Script
**Date: November 28, 2024**

A PsychoPy implementation of a probabilistic learning task where participants learn to select between two symbols with different reward probabilities.

## Trial Structure:
- Decision phase: 1.25s 
- First delay: 1.5-2s
- Feedback: 0.75s
- Inter-trial interval: 2-3s

## Key Features:
- Windowed mode (1000x800px)
- Comprehensive instructions
- Easy termination with 'q' key
- Data saved automatically as CSV
- 80 trials per block
- 70/30 reward probability structure

## Required Files:
Place in stim/ folder:
- A.png, B.png, C.png (symbols)
- win.png, loss.png, NEU.png (feedback)
- fix.png (fixation cross)

In [2]:

from psychopy import visual, core, data, event, gui
import pandas as pd
import numpy as np
from pathlib import Path

class ProbabilisticLearningTask:
    def __init__(self):
        # Task settings 
        self.settings = {
            'n_trials': 80,  # Trials per block
            'decision_duration': 1.25,  # Decision phase duration
            'delay_min': 1.5,  # Minimum delay duration
            'delay_max': 2.0,  # Maximum delay duration
            'feedback_duration': 0.75,  # Feedback duration
            'iti_min': 2.0,  # Minimum inter-trial interval
            'iti_max': 3.0,  # Maximum inter-trial interval
            'win_probability': 0.70  # Probability of win for correct choice
        }
        
        # Setup paths
        self.base_path = Path.cwd()
        self.stim_path = self.base_path / 'stimuli'
        self.data_path = self.base_path / 'data'
        self.data_path.mkdir(exist_ok=True)
        
        # Define stimuli filenames
        self.stimuli = {
            'symbols': ['A.png', 'B.png', 'C.png'],
            'fixation': 'fix.png',
            'feedback': {
                'win': 'win.png',
                'loss': 'loss.png',
                'neutral': 'NEU.png'
            }
        }

        # Task instructions
        self.instructions = [
            """Welcome to the experiment!

In this task, you will see two symbols on each trial.
Your goal is to learn which symbol is more likely to give you rewards.

Press SPACE to continue...""",

            """On each trial:
1. Two symbols will appear
2. Choose the left symbol with the LEFT ARROW key
3. Choose the right symbol with the RIGHT ARROW key
4. You have 1.25 seconds to make your choice

Press SPACE to continue...""",

            """After your choice:
- If you see a WIN image, you earned points
- If you see a LOSS image, you lost points
- If you see a NEUTRAL image, no points change

The same symbols will have different probabilities of winning.
These probabilities may change during the task.

Press SPACE to continue...""",

            """Important Notes:
- Respond as quickly and accurately as possible
- If you don't respond in time, you'll see a neutral outcome
- You can press 'q' at any time to end the experiment
- Your data will be saved automatically

Press SPACE to begin the task."""
        ]

    def setup_experiment(self):
        """Initialize PsychoPy window and load stimuli"""
        # Create window (not fullscreen)
        self.win = visual.Window(
            size=[1000, 800],  # Reduced window size
            fullscr=False,     # Windowed mode
            units='height',
            color=[0, 0, 0],
            allowGUI=True      # Allow GUI for easy closing
        )
        
        # Load stimuli
        self.stim = {
            'symbols': [
                visual.ImageStim(self.win, image=str(self.stim_path / img))
                for img in self.stimuli['symbols']
            ],
            'fixation': visual.ImageStim(
                self.win, 
                image=str(self.stim_path / self.stimuli['fixation'])
            ),
            'feedback': {
                key: visual.ImageStim(self.win, image=str(self.stim_path / path))
                for key, path in self.stimuli['feedback'].items()
            },
            'text': visual.TextStim(
                self.win,
                text='',
                height=0.05,
                wrapWidth=0.8,
                color='white'
            )
        }

    def show_instructions(self):
        """Display task instructions"""
        for instruction in self.instructions:
            self.stim['text'].text = instruction
            self.stim['text'].draw()
            self.win.flip()
            
            # Wait for space key or check for quit
            while True:
                keys = event.waitKeys(keyList=['space', 'q'])
                if 'q' in keys:
                    self.win.close()
                    core.quit()
                if 'space' in keys:
                    break

    def get_participant_info(self):
        """Show dialog to collect participant information"""
        exp_info = {
            'participant': '',
            'session': '001', 
            'run': '1',
        }

        dlg = gui.DlgFromDict(
            dictionary=exp_info,
            title='Task Info',
            fixed=['session']  # Make session field unchangeable
        )

        if dlg.OK:
            return exp_info
        else:
            core.quit()  # User hit cancel
                    
    def run_trial(self):
        """Run a single trial"""
        # Check for quit key
        if 'q' in event.getKeys(keyList=['q']):
            self.win.close()
            core.quit()

        # 1. Decision Phase (1.25s)
        positions = [(-0.15, 0), (0.15, 0)]
        np.random.shuffle(positions)
        
        for sym, pos in zip(self.stim['symbols'][:2], positions):
            sym.pos = pos
            sym.draw()
        self.win.flip()
        
        # Get response with timeout
        timer = core.Clock()
        keys = event.waitKeys(
            maxWait=self.settings['decision_duration'],
            keyList=['left', 'right', 'q'],
            timeStamped=timer
        )
        
        # Check for quit
        if keys and 'q' in keys[0]:
            self.win.close()
            core.quit()

        # 2. Delay Period (1.5-2s)
        delay_time = np.random.uniform(
            self.settings['delay_min'],
            self.settings['delay_max']
        )
        self.stim['fixation'].draw()
        self.win.flip()
        core.wait(delay_time)
        
        # 3. Feedback (0.75s)
        if not keys:  # No response
            feedback = 'neutral'
            rt = None
            choice = None
            correct = False
        else:
            key, rt = keys[0]
            choice = 0 if key == 'left' else 1
            
            # Determine outcome
            correct = choice == 0  # First symbol is always correct
            if correct:
                outcome = np.random.random() < self.settings['win_probability']
            else:
                outcome = np.random.random() < (1 - self.settings['win_probability'])
            
            feedback = 'win' if outcome else 'loss'
        
        self.stim['feedback'][feedback].draw()
        self.win.flip()
        core.wait(self.settings['feedback_duration'])
        
        # 4. Inter-trial interval (2-3s)
        iti_time = np.random.uniform(
            self.settings['iti_min'],
            self.settings['iti_max']
        )
        self.stim['fixation'].draw()
        self.win.flip()
        core.wait(iti_time)
        
        return {
            'rt': rt,
            'choice': choice,
            'correct': correct,
            'feedback': feedback
        }

    def run_experiment(self):
        """Run the complete experiment"""
        # Setup
        participant_info = self.get_participant_info()
        self.setup_experiment()
        
        # Show instructions
        self.show_instructions()
        
        # Create data storage
        data = []
        
        try:
            # Run trials
            for trial in range(self.settings['n_trials']):
                # Run trial and save data
                trial_data = self.run_trial()
                trial_data.update({
                    'trial': trial,
                    'participant': participant_info['participant'],
                    'session': participant_info['session']
                })
                data.append(trial_data)
                
        finally:
            # Always save data, even if experiment is terminated early
            if data:  # Only save if there's data to save
                df = pd.DataFrame(data)
                filename = f"sub-{participant_info['participant']}_ses-{participant_info['session']}_task-problearn.csv"
                df.to_csv(self.data_path / filename, index=False)
            
            # Clean up
            self.win.close()

if __name__ == "__main__":
    task = ProbabilisticLearningTask()
    task.run_experiment()


51.4214 	WARNING 	Monitor specification not found. Creating a temporary one...


SystemExit: 0

c:\program files\psychopy3\lib\site-packages\IPython\core\interactiveshell.py:3299: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## This modified script:

1. Integrates LSL to receive R-peak markers in real-time
2. Creates precise timing bins (10ms) for the entire cardiac cycle
3. Allows three presentation modes:
   - **Systole** (0-300ms post R-peak)
   - **Diastole** (300ms to next R-peak)
   - **Mixed** (random selection across the whole cycle)
4. Records detailed timing information:
   - R-peak times
   - Feedback presentation times
   - Cardiac phase
   - Time from R-peak
   - Timing mode for each block

### To use this script:

1. Make sure you have `pylsl` installed:
   ```bash
   pip install pylsl

2. Set up your ECG data streaming system to send LSL markers for R-peaks
3. The script will:
    - Wait for LSL stream
    - Create timing bins for each cardiac cycle
    - Present feedback at cardiac-specific times
    - Save detailed timing information in the output data

The logged data will include all cardiac timing information for detailed analysis of cardiac effects on learning.


In [1]:
from psychopy import visual, core, data, event, gui
import pandas as pd
import numpy as np
from pathlib import Path
from pylsl import StreamInlet, resolve_stream
import threading
import queue
import time

class CardiacSyncedLearningTask:
    def __init__(self):
        # Task settings 
        self.settings = {
            'n_trials': 80,  # Trials per block
            'decision_duration': 1.25,  # Decision phase duration
            'delay_min': 1.5,  # Minimum delay duration
            'delay_max': 2.0,  # Maximum delay duration
            'feedback_duration': 0.75,  # Feedback duration
            'iti_min': 2.0,  # Minimum inter-trial interval
            'iti_max': 3.0,  # Maximum inter-trial interval
            'win_probability': 0.70,  # Probability of win for correct choice
            'systole_duration': 0.300,  # 300ms post R-peak
            'time_resolution': 0.010  # 10ms bins
        }
        
        # LSL and timing variables
        self.r_peak_times = queue.Queue()
        self.current_r_peak = None
        self.next_r_peak = None
        self.cardiac_timing_options = {
            'systole': [],  # Will store systole timepoints
            'diastole': []  # Will store diastole timepoints
        }
        
        # Initialize paths and stimuli as before...
        self.setup_paths_and_stimuli()
        
        # Timing mode for the block
        self.timing_modes = ['systole', 'diastole', 'mixed']
        
    def setup_lsl(self):
        """Setup LSL inlet for R-peak markers"""
        print("Looking for R-peak markers stream...")
        streams = resolve_stream('type', 'R_PEAK')
        self.inlet = StreamInlet(streams[0])
        
        # Start R-peak collection thread
        self.lsl_thread = threading.Thread(target=self.collect_r_peaks)
        self.lsl_thread.daemon = True
        self.lsl_thread.start()
        
    def collect_r_peaks(self):
        """Continuously collect R-peak markers"""
        while True:
            sample, timestamp = self.inlet.pull_sample()
            self.r_peak_times.put(timestamp)
            
    def get_cardiac_timing_options(self, r_peak_time, next_r_peak_time):
        """Generate timing options relative to R-peak"""
        options = {'systole': [], 'diastole': []}
        current_time = r_peak_time
        
        while current_time < next_r_peak_time:
            delay = current_time - r_peak_time
            if delay <= self.settings['systole_duration']:
                options['systole'].append(current_time)
            else:
                options['diastole'].append(current_time)
            current_time += self.settings['time_resolution']
            
        return options
        
    def select_feedback_time(self, timing_mode):
        """Select feedback presentation time based on mode"""
        options = []
        if timing_mode == 'systole':
            options = self.cardiac_timing_options['systole']
        elif timing_mode == 'diastole':
            options = self.cardiac_timing_options['diastole']
        else:  # mixed
            options = (self.cardiac_timing_options['systole'] + 
                      self.cardiac_timing_options['diastole'])
            
        return np.random.choice(options)
        
    def run_trial(self, timing_mode):
        """Run a single trial with cardiac-synced feedback"""
        # Decision phase remains same...
        
        # Get response with timeout
        response_data = self.get_response()
        if response_data['quit']:
            return None
            
        # Wait for next R-peak and prepare feedback timing
        r_peak = self.r_peak_times.get()
        next_r_peak = self.r_peak_times.get()
        
        # Generate timing options for this cardiac cycle
        self.cardiac_timing_options = self.get_cardiac_timing_options(
            r_peak, next_r_peak
        )
        
        # Select feedback time
        feedback_time = self.select_feedback_time(timing_mode)
        
        # Wait until selected feedback time
        while time.time() < feedback_time:
            core.wait(0.001)  # Small wait to prevent CPU overload
            
        # Present feedback
        feedback_data = self.show_feedback(response_data)
        
        # Record timing information
        timing_data = {
            'r_peak_time': r_peak,
            'next_r_peak_time': next_r_peak,
            'feedback_time': feedback_time,
            'cardiac_phase': ('systole' if feedback_time - r_peak <= 
                            self.settings['systole_duration'] else 'diastole'),
            'time_from_r_peak': feedback_time - r_peak
        }
        
        # Combine all trial data
        trial_data = {**response_data, **feedback_data, **timing_data}
        
        return trial_data

    def run_block(self, timing_mode):
        """Run a block with specified cardiac timing mode"""
        block_data = []
        
        for trial in range(self.settings['n_trials']):
            trial_data = self.run_trial(timing_mode)
            if trial_data is None:  # Quit signal
                break
            block_data.append(trial_data)
            
        return block_data

    def run_experiment(self):
        """Run the complete experiment"""
        # Setup LSL first
        self.setup_lsl()
        
        # Get some initial R-peaks to ensure stable recording
        print("Calibrating R-peak detection...")
        core.wait(3.0)
        
        # Run blocks with different timing modes
        all_data = []
        for mode in self.timing_modes:
            print(f"Starting {mode} block...")
            block_data = self.run_block(mode)
            for trial in block_data:
                trial['timing_mode'] = mode
            all_data.extend(block_data)
            
        # Save data
        df = pd.DataFrame(all_data)
        self.save_data(df)


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
